### Prepare files
Code to prepare .gjf and .pbs files to run on GADI supercomputer.

In [82]:
#libraries and packages needed:
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns
import pandas as pd
import numpy as np
import shutil
import glob
import csv
import ast
import os
import re
#from mendeleev import element

In [83]:
run = 1
geom_version = 'new' # 'old'

lots = ['B971']
bss = ['6-31Gs']

# lots = ['B971', 'B3LYP', 'wB97XD', 'M062X', 'PBE1PBE', 'TPSSh', 'B2PLYP', 'B2PLYPD']
# bss = ['6-31Gs', '6-31Gss', 'pc-1', 'pcseg-1', 'AUG-pcseg-1', 'pc-2', 'pcseg-2', 'AUG-pcseg-2',
#        'Def2SVP', 'Def2SVPD', 'Def2TZVP', 'Def2TZVPD', 'cc-pVDZ', 'AUG-cc-pVDZ', 'cc-pVTZ']

#proper bs names:
bss_s = ['6-31G*', '6-31G**', 'pc-1', 'pcseg-1', 'AUG-pcseg-1', 'pc-2', 'pcseg-2', 'AUG-pcseg-2', 
       'Def2SVP', 'Def2SVPD', 'Def2TZVP', 'Def2TZVPD', 'cc-pVDZ', 'AUG-cc-pVDZ', 'cc-pVTZ']

u_bss = ['pc-1', 'pcseg-1', 'AUG-pc-1', 'AUG-pcseg-1', 'SNSD', 'Def2TZVPD', 'Def2TZVPPD', 'pc-2', 'pcseg-2', 'AUG-pc-2', 'AUG-pcseg-2']

two_mult = ['CH','HCO','NO2','OH','SH','BO','ClO','CN','HO2']

files_names = glob.glob(f'\\Users\\natia\\OneDrive\\Pulpit\\UNI\\YEAR 4\\PROJECT\\benchmarking\\initial geometries\\{geom_version}\\*.xyz')
template_path = '\\Users\\natia\\OneDrive\\Pulpit\\UNI\\YEAR 4\\PROJECT\\benchmarking\\templates\\VPT2_gaussian.gjf'
a_template_path = '\\Users\\natia\\OneDrive\\Pulpit\\UNI\\YEAR 4\\PROJECT\\benchmarking\\templates\\VPT2_gaussian_SymmLoose.gjf'


In [84]:
#define functions:

def run1(mols_considered, path) :
                
    if not os.path.exists(f'{path}\\{lot}\\{bs}') :

          os.makedirs(f'{path}\\{lot}\\{bs}')

    with open('\\Users\\natia\\OneDrive\\Pulpit\\UNI\\YEAR 4\\PROJECT\\benchmarking\\harmonic_benchmark.csv', 'r', encoding = 'latin1') as ref_atoms :

        next(ref_atoms)

        for row in csv.reader(ref_atoms) :

            mols_considered.append(row[0])

    mols_considered = list(set(mols_considered))
    
def run2(mols_considered, path) :
                    
    files = glob.glob(f'{path}\\{lot}\\{bs}\\*.csv')

    for file in files :

        with open(file, 'r', encoding='utf-8') as current_file:

            df = pd.read_csv(file, usecols = ['molecule'])
            molecules = df[df.columns[0]].to_numpy()

            for molecule in molecules : 

                mols_considered.append(molecule)

            df = pd.read_csv(file, usecols = ['type of error'])
            error_type = df[df.columns[0]].to_numpy()

            for i in error_type :

                c = c + 1

                if i == '*****' :
                    df = pd.read_csv(file, usecols = ['molecule'])
                    mol = df.iloc[c][0]
                    a_mols.append(mol)
                    
def generate_gjf_for_non_gauss_bs(path) :
    
    # copy the template file, change its name to the one for the molecule being considered
    # https://thispointer.com/python-how-to-copy-files-from-one-location-to-another-using-shutil-copy/

    shutil.copy(template, f'{path}\\{lot}\\{bs}\\{filen.rstrip()}_{lot}_{bs}.gjf')

    basis_data = [] 
    elements_list = []
    atomic_numbers = []
    basis_str = ''

    # store the element composition in the molecular formula including the atomic numbers

    for letter in re.findall('[A-Z][^A-Z]*', filen.rstrip()) :

        symbol = re.split(r'[0-9]', letter.split()[0])[0]
        poss_under = re.split(r'(_)', symbol.split()[0])[0]
        elements_list.append(poss_under)

    for atom in elements_list :

        symbol = element(atom)
        atomic_numbers.append(symbol.atomic_number)

    # with the atomic number from the molecular formula, store the corresponding elements from the user-defined basis set

    basis_file = open(f'\\Users\\natia\\OneDrive\\Pulpit\\UNI\\YEAR 4\\PROJECT\\benchmarking\\python code\\user_basis\\{bs}.txt', 'r')
    contents = basis_file.read()
    basis_dict = ast.literal_eval(contents)

    for number in atomic_numbers :

        if number in basis_dict:
            basis_data.append(basis_dict[number])

    basis_str = '\n'.join(basis_data)

    # save the info for the respective geometry into the geom_data list

    with open(file,'r') as inp_geom :

        geom_data = inp_geom.read()
        geom_data = geom_data.split('\n',2)[-1]
        inp_geom.close()

    # replace the selected keywords with the corresponding values
    # https://www.kite.com/python/answers/how-to-update-and-replace-text-in-a-file-in-python

    with open(f'{path}\\{lot}\\{bs}\\{filen.rstrip()}_{lot}_{bs}.gjf','r+') as inp_template:

        template_data = inp_template.read()
        template_data = re.sub('{cpu}', '8', template_data)
        template_data = re.sub('{mem}', '32', template_data)
        template_data = re.sub('{geometry}', geom_data, template_data)
        template_data = re.sub('{LT}', lot, template_data)
        template_data = re.sub('{basis}', 'Gen', template_data)
        template_data = re.sub('{molecule}', filen.rstrip(), template_data)
        template_data = re.sub('{charge}', '0', template_data)
        template_data = re.sub('{basis_data}', basis_str, template_data)

        # consider different values of multiplicity

        if filen == 'tri-CH2' :

            template_data = re.sub('{multiplicity}', '3', template_data)

        if filen == 'NH' :

            template_data = re.sub('{multiplicity}', '3', template_data)

        if filen == 'PH' :

            template_data = re.sub('{multiplicity}', '3', template_data)

        if filen in two_mult :

            template_data = re.sub('{multiplicity}', '2', template_data)

        else :

            template_data = re.sub('{multiplicity}', '1', template_data)

        inp_template.seek(0)
        inp_template.write(template_data)
        inp_template.truncate()
    
def generate_gjf_for_gauss_bs(path) :
            
        # copy the template file, change its name to the one for the molecule being considered
        # https://thispointer.com/python-how-to-copy-files-from-one-location-to-another-using-shutil-copy/

        shutil.copy(template, f'{path}\\{lot}\\{bs}\\{filen.rstrip()}_{lot}_{bs}.gjf')

        # save the information for the respective geometry into the geom_data list

        with open(file,'r') as inp_geom:

            geom_data = inp_geom.read()
            geom_data = geom_data.split('\n',2)[-1]
            inp_geom.close()

        # replace the selected keywords with the corresponding values
        # https://www.kite.com/python/answers/how-to-update-and-replace-text-in-a-file-in-python

        with open(f'{path}\\{lot}\\{bs}\\{filen.rstrip()}_{lot}_{bs}.gjf','r+') as inp_template :

            template_data = inp_template.read()
            template_data = re.sub('{cpu}', '8', template_data)
            template_data = re.sub('{mem}', '32', template_data)
            template_data = re.sub('{geometry}', geom_data, template_data)
            template_data = re.sub('{LT}', lot, template_data)
            template_data = re.sub('{basis}', bs_s, template_data)
            template_data = re.sub('{molecule}', filen.rstrip(), template_data)
            template_data = re.sub('{charge}', '0', template_data)

            # consider different values of multiplicity

            if filen == 'tri-CH2' :

                template_data = re.sub('{multiplicity}', '3', template_data)

            if filen == 'NH' :

                template_data = re.sub('{multiplicity}', '3', template_data)

            if filen == 'PH' :

                template_data = re.sub('{multiplicity}', '3', template_data)

            if filen in two_mult :

                template_data = re.sub('{multiplicity}', '2', template_data)

            else :

                template_data = re.sub('{multiplicity}', '1', template_data)

            # writes the new templates

            inp_template.seek(0)
            inp_template.write(template_data)
            inp_template.truncate()

def generate_pbs(path) :

    shutil.copy('\\Users\\natia\\OneDrive\\Pulpit\\UNI\\YEAR 4\\PROJECT\\benchmarking\\templates\\submission.pbs', f'{path}\\{lot}\\{bs}\\{filen.rstrip()}_{lot}_{bs}.pbs')
    
    with open(f'{path}\\{lot}\\{bs}\\{filen.rstrip()}_{lot}_{bs}.pbs', 'r+') as inp_sub :
        
        submission_data = inp_sub.read()
        submission_data = re.sub('{file_name}', f'{filen.rstrip()}_{lot}_{bs}', submission_data)
        submission_data = re.sub('{queue}', 'normal', submission_data)
        submission_data = re.sub('{time}', '48:00:00', submission_data)
        submission_data = re.sub('{mem}', '32', submission_data)
        submission_data = re.sub('{jobfs}', '32', submission_data)
        submission_data = re.sub('{cpus}', '8', submission_data)
        
        inp_sub.seek(0)
        inp_sub.write(submission_data)
        inp_sub.truncate()

In [85]:
for lot in lots :
    
    for (bs, bs_s) in zip(bss, bss_s) :
        
        mols_considered = []
        a_mols = []
        c = -1
        path = '\\Users\\natia\\OneDrive\\Pulpit\\UNI\\YEAR 4\\PROJECT\\benchmarking\\calculations\\input'

        if run == 1 : #prepare files for run 1

            run1(mols_considered, path)
            
        elif run == 2 : #prepare files for run 2

            path = '\\Users\\natia\\OneDrive\\Pulpit\\UNI\\YEAR 4\\PROJECT\\benchmarking\\calculations\\failed\\run1'
            run2(mols_considered, path)
                    
        # store the number of atoms in each molecule (will be useful when splitting the list of conformers)

        for file in files_names:

            # store name without the CONF#

            filen = file.split('\\')[-1].split('.xyz')[0]

            if filen in mols_considered:

                if filen in a_mols : 
                    
                    template = a_template_path
                
                else : 
                    
                    template = template_path

                if bs not in u_bss:

                    generate_gjf_for_gauss_bs(path)

                else :

                    generate_gjf_for_non_gauss_bs(path)


                generate_pbs(path)